# More attempts to understand what is going on with appending zarrs

https://github.com/pydata/xarray/issues/5878


In [97]:
import fsspec
import xarray as xr
import json
import gcsfs

## define a mapper to the ldeo-glaciology bucket
### needs a token
with open('/Users/jkingslake/Documents/misc/ldeo-glaciology-bc97b12df06b.json') as token_file:
    token = json.load(token_file)

filename = 'gs://ldeo-glaciology/append_test/test59'
    
mapper = fsspec.get_mapper(filename, mode='w', token=token)

## define two simple datasets
ds0 = xr.Dataset({'temperature': (['time'],  [50, 51, 52])}, coords={'time': [1, 2, 3]})
ds1 = xr.Dataset({'temperature': (['time'],  [53, 54, 55])}, coords={'time':  [4, 5, 6]})

## write the first ds to bucket
ds0.to_zarr(mapper)



#!gsutil setmeta -h -r "Cache-Control:no-store" gs://ldeo-glaciology/append_test/test53**




In [98]:
## append the second ds to the same zarr store
ds1.to_zarr(mapper, mode='a', append_dim='time')

In [76]:


## load the zarr store
ds_both = xr.open_zarr(mapper)

## this is 3, indicating that the append did not work 
len(ds_both.time)

3

In [68]:
!gsutil setmeta -h "Cache-Control:no-store" gs://ldeo-glaciology/append_test/test53/**

zsh:1: no matches found: gs://ldeo-glaciology/append_test/test53/**


In [102]:
ds = xr.open_dataset('gs://ldeo-glaciology/append_test/test59', engine='zarr', consolidated=False)
ds

<xarray.Dataset>
Dimensions:      (time: 3)
Coordinates:
  * time         (time) int64 1 2 3
Data variables:
    temperature  (time) int64 ...

In [40]:
import gcsfs
fs = gcsfs.GCSFileSystem()
print(fs.cat('gs://ldeo-glaciology/append_test/test50/temperature/.zarray').decode())


{
    "chunks": [
        3
    ],
    "compressor": {
        "blocksize": 0,
        "clevel": 5,
        "cname": "lz4",
        "id": "blosc",
        "shuffle": 1
    },
    "dtype": "<i8",
    "fill_value": null,
    "filters": null,
    "order": "C",
    "shape": [
        6
    ],
    "zarr_format": 2
}


In [58]:
import zarr
zarr.consolidate_metadata(mapper) 

<zarr.hierarchy.Group '/'>

In [107]:
#gs://ldeo-glaciology/GL_apres_2022_zarr_test_05   'gs://ldeo-glaciology/GL_apres_2022_zarr_test_large_3'
!curl https://storage.googleapis.com/ldeo-glaciology/GL_apres_2022_zarr_test_large_3/time/.zarray

{
    "chunks": [
        94
    ],
    "compressor": {
        "blocksize": 0,
        "clevel": 5,
        "cname": "lz4",
        "id": "blosc",
        "shuffle": 1
    },
    "dtype": "<i8",
    "fill_value": null,
    "filters": null,
    "order": "C",
    "shape": [
        658
    ],
    "zarr_format": 2
}

In [38]:
!curl https://storage.googleapis.com/ldeo-glaciology/append_test/test50/temperature/.zarray

{
    "chunks": [
        3
    ],
    "compressor": {
        "blocksize": 0,
        "clevel": 5,
        "cname": "lz4",
        "id": "blosc",
        "shuffle": 1
    },
    "dtype": "<i8",
    "fill_value": null,
    "filters": null,
    "order": "C",
    "shape": [
        9
    ],
    "zarr_format": 2
}

In [114]:
ds_reloaded = xr.open_dataset('gs://ldeo-glaciology/GL_apres_2022_zarr_test_large_3', engine='zarr', consolidated=False, chunks='auto')
ds_reloaded

<xarray.Dataset>
Dimensions:          (attenuator_setting_pair: 2, time: 658, chirp_time: 40001,
                      chirp_num: 20, profile_range: 6658)
Coordinates:
    AFGain           (attenuator_setting_pair) int64 dask.array<chunksize=(2,), meta=np.ndarray>
    attenuator       (attenuator_setting_pair) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    burst_number     (time) int64 dask.array<chunksize=(658,), meta=np.ndarray>
  * chirp_num        (chirp_num) int64 0 1 2 3 4 5 6 7 ... 13 14 15 16 17 18 19
  * chirp_time       (chirp_time) timedelta64[ns] 00:00:00 ... 00:00:01
    filename         (time) <U83 dask.array<chunksize=(658,), meta=np.ndarray>
  * profile_range    (profile_range) float64 0.0 0.2103 ... 1.4e+03 1.4e+03
  * time             (time) datetime64[ns] 2022-05-27T15:06:30 ... 2022-09-10...
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    battery_voltage  (time) float64 dask.array<chunksize=(658,), meta=np.ndarray>
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 dask.array<chunksize=(36, 15003, 15, 2), meta=np.ndarray>
    latitude         (time) float64 dask.array<chunksize=(658,), meta=np.ndarray>
    longitude        (time) float64 dask.array<chunksize=(658,), meta=np.ndarray>
    profile          (time, profile_range, chirp_num, attenuator_setting_pair) complex128 dask.array<chunksize=(36, 4995, 15, 2), meta=np.ndarray>
    temperature_1    (time) float64 dask.array<chunksize=(658,), meta=np.ndarray>
    temperature_2    (time) float64 dask.array<chunksize=(658,), meta=np.ndarray>

In [113]:
ds_reloaded.nbytes/1e9

9.623244456

In [120]:
ds_reloaded = xr.open_dataset('gs://ldeo-glaciology/GL_apres_2022_zarr_test_large_3', engine='zarr', consolidated=True, chunks='auto')
ds_reloaded

<xarray.Dataset>
Dimensions:          (attenuator_setting_pair: 2, time: 564, chirp_time: 40001,
                      chirp_num: 20, profile_range: 6658)
Coordinates:
    AFGain           (attenuator_setting_pair) int64 dask.array<chunksize=(2,), meta=np.ndarray>
    attenuator       (attenuator_setting_pair) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    burst_number     (time) int64 dask.array<chunksize=(564,), meta=np.ndarray>
  * chirp_num        (chirp_num) int64 0 1 2 3 4 5 6 7 ... 13 14 15 16 17 18 19
  * chirp_time       (chirp_time) timedelta64[ns] 00:00:00 ... 00:00:01
    filename         (time) <U83 dask.array<chunksize=(564,), meta=np.ndarray>
  * profile_range    (profile_range) float64 0.0 0.2103 ... 1.4e+03 1.4e+03
  * time             (time) datetime64[ns] 2022-05-27T15:06:30 ... 2022-08-23...
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    battery_voltage  (time) float64 dask.array<chunksize=(564,), meta=np.ndarray>
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 dask.array<chunksize=(36, 15003, 15, 2), meta=np.ndarray>
    latitude         (time) float64 dask.array<chunksize=(564,), meta=np.ndarray>
    longitude        (time) float64 dask.array<chunksize=(564,), meta=np.ndarray>
    profile          (time, profile_range, chirp_num, attenuator_setting_pair) complex128 dask.array<chunksize=(36, 4995, 15, 2), meta=np.ndarray>
    temperature_1    (time) float64 dask.array<chunksize=(564,), meta=np.ndarray>
    temperature_2    (time) float64 dask.array<chunksize=(564,), meta=np.ndarray>

In [122]:
ds_reloaded = xr.open_dataset('gs://ldeo-glaciology/GL_apres_2022_zarr_test_small_1', engine='zarr', consolidated=False, chunks='auto')
ds_reloaded

<xarray.Dataset>
Dimensions:          (attenuator_setting_pair: 2, time: 4, chirp_time: 40001,
                      chirp_num: 20, profile_range: 6658)
Coordinates:
    AFGain           (attenuator_setting_pair) int64 dask.array<chunksize=(2,), meta=np.ndarray>
    attenuator       (attenuator_setting_pair) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    burst_number     (time) int64 dask.array<chunksize=(4,), meta=np.ndarray>
  * chirp_num        (chirp_num) int64 0 1 2 3 4 5 6 7 ... 13 14 15 16 17 18 19
  * chirp_time       (chirp_time) timedelta64[ns] 00:00:00 ... 00:00:01
    filename         (time) <U83 dask.array<chunksize=(4,), meta=np.ndarray>
  * profile_range    (profile_range) float64 0.0 0.2103 ... 1.4e+03 1.4e+03
  * time             (time) datetime64[ns] 2022-05-27T15:06:30 ... 2022-06-22...
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    battery_voltage  (time) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 dask.array<chunksize=(4, 40001, 20, 2), meta=np.ndarray>
    latitude         (time) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    longitude        (time) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    profile          (time, profile_range, chirp_num, attenuator_setting_pair) complex128 dask.array<chunksize=(4, 6658, 20, 2), meta=np.ndarray>
    temperature_1    (time) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    temperature_2    (time) float64 dask.array<chunksize=(4,), meta=np.ndarray>

In [39]:
ds2 = xr.Dataset({'temperature': (['time'],  [53, 54, 55])}, coords={'time':  [7, 8, 9]})

ds2.to_zarr(mapper, mode='a', append_dim='time')


In [117]:
!curl https://storage.googleapis.com/ldeo-glaciology/GL_apres_2022_zarr_test_small_1/time/.zarray

{
    "chunks": [
        2
    ],
    "compressor": {
        "blocksize": 0,
        "clevel": 5,
        "cname": "lz4",
        "id": "blosc",
        "shuffle": 1
    },
    "dtype": "<i8",
    "fill_value": null,
    "filters": null,
    "order": "C",
    "shape": [
        4
    ],
    "zarr_format": 2
}

In [ ]:
import fsspec
import xarray as xr
import json
import gcsfs

## define a mapper to the ldeo-glaciology bucket
### needs a token
with open('/Users/jkingslake/Documents/misc/ldeo-glaciology-bc97b12df06b.json') as token_file:
    token = json.load(token_file)

filename = 'gs://ldeo-glaciology/append_test/test57'
    
mapper = fsspec.get_mapper(filename, mode='w', token=token)

## define two simple datasets
ds0 = xr.Dataset({'temperature': (['time'],  [50, 51, 52])}, coords={'time': [1, 2, 3]})
ds1 = xr.Dataset({'temperature': (['time'],  [53, 54, 55])}, coords={'time':  [4, 5, 6]})

## write the first ds to bucket
ds0.to_zarr(mapper)
